In [1]:
import pandas as pd
import numpy as np
import glob

from finrl.meta.preprocessor.preprocessors import FeatureEngineer, data_split

In [ ]:
import os
import glob

# Путь к папке с данными
data_dir = "../data"

# Шаблон для поиска файлов в папке data
file_pattern = os.path.join(data_dir, "*.csv")  # Ищем все CSV файлы в папке data

# Поиск файлов
file_paths = glob.glob(file_pattern)

# Вывод найденных файлов
print("Найденные файлы:", file_paths)

In [ ]:
import pandas as pd
import glob
import os

# Путь к папке с данными
data_dir = "../data"

# Создаем пустой список для хранения данных
data_frames = []

# Поиск всех CSV-файлов в папке data
file_pattern = os.path.join(data_dir, "*.csv")  # Шаблон для поиска файлов
file_paths = glob.glob(file_pattern)

# Обрабатываем каждый файл
for file_path in file_paths:
    # Определяем тикер из имени файла (без расширения)
    file_name = os.path.basename(file_path)  # Имя файла без пути
    symbol = file_name.split("_")[0]  # Берем часть до первого "_"

    try:
        # Читаем CSV-файл
        df = pd.read_csv(file_path)

        # Очищаем имена столбцов (убираем пробелы и приводим к нижнему регистру)
        df.columns = df.columns.str.strip().str.lower()

        # Переименовываем столбцы
        df.rename(columns={
            "open time": "date",
            "open": "open",
            "high": "high",
            "low": "low",
            "close": "close",
            "volume": "volume"
        }, inplace=True)

        # Добавляем столбец tic
        df["tic"] = symbol

        # Проверяем наличие необходимых столбцов
        required_columns = ["date", "open", "high", "low", "close", "volume"]
        if all(col in df.columns for col in required_columns):
            df = df[required_columns + ["tic"]]
        else:
            print(f"В файле {file_path} отсутствуют необходимые столбцы.")
            print("Доступные столбцы:", df.columns)
            continue

        # Добавляем DataFrame в список
        data_frames.append(df)

    except Exception as e:
        print(f"Ошибка при обработке файла {file_path}: {e}")

# Объединяем все DataFrame в один
if data_frames:  # Проверяем, что список не пустой
    final_df = pd.concat(data_frames, ignore_index=True)

    # Преобразуем столбец date в формат datetime
    final_df["date"] = pd.to_datetime(final_df["date"])

    # Выводим первые строки итогового DataFrame
    print(final_df.head())
    print("Размер итогового DataFrame:", final_df.shape)
else:
    print("Нет данных для объединения.")


In [ ]:
final_df['tic'].unique()

In [5]:
symbols = ['JUPUSDT', 'PEPEUSDT', 'APTUSDT', 'CAKEUSDT', 'HBARUSDT', 'STRKUSDT', 'USDCUSDT']

In [ ]:
df_prep = final_df[final_df['tic'].isin(symbols)]

max_start_date = df_prep.groupby('tic')['date'].min().max()

df_prep_aligned = df_prep[df_prep['date'] >= max_start_date]

print("Aligned start date:", max_start_date)
print(df_prep_aligned.groupby('tic')['date'].min())  # Должны быть одинаковые даты для всех тикеров
print(df_prep_aligned.shape)

In [ ]:
df_prep_aligned

## Preprocess Data

In [ ]:
fe = FeatureEngineer(
                    use_technical_indicator=True,
                    use_turbulence=False,
                    user_defined_feature = False)

df = fe.preprocess_data(df_prep_aligned)
print(df.shape)
print(df['tic'].unique())

In [ ]:
# Add covariance matrix as states
# add covariance matrix as states
df = df.sort_values(['date','tic'], ignore_index=True)
df.index = df.date.factorize()[0]

cov_list = []
return_list = []

# look back
lb_val = 24
lookback = lb_val
for i in range(lookback,len(df.index.unique())):
  data_lookback = df.loc[i-lookback:i,:]
  price_lookback = data_lookback.pivot_table(index = 'date',columns = 'tic', values = 'close')
  return_lookback = price_lookback.pct_change().dropna()
  return_list.append(return_lookback)

  covs = return_lookback.cov().values 
  cov_list.append(covs)
  
df_cov = pd.DataFrame({'date':df.date.unique()[lookback:],'cov_list':cov_list,'return_list':return_list})
df = df.merge(df_cov, on='date')
df = df.sort_values(['date','tic']).reset_index(drop=True)
print(df.shape)
df.head()

## Design Environment

In [ ]:
# Определяем границы для обучающей выборки
start_train = df['date'].min()  # Самая ранняя дата в данных
end_train = df['date'].quantile(0.8)  # 80% данных для обучения (по времени)

print("Start of training period:", start_train)
print("End of training period:", end_train)

In [ ]:
# Определяем границы для тестовой выборки
start_test = end_train  # Начало тестовой выборки = конец обучающей
end_test = df['date'].max()  # Самая поздняя дата в данных

print("Start of testing period:", start_test)
print("End of testing period:", end_test)

In [ ]:
train = data_split(df, start_train, end_train)
train.head()

https://www.kaggle.com/code/learnmore1/deep-reinforcement-learning-for-stock-trading-1#Part-5.-Design-Environment

In [13]:
import numpy as np
import pandas as pd
import gymnasium as gym
from gymnasium import spaces
from gymnasium.utils import seeding
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3 import DDPG
from finrl.agents.stablebaselines3.models import DRLAgent


class StockPortfolioEnv(gym.Env):
    """A single stock trading environment for OpenAI gym

    Attributes
    ----------
        df: DataFrame
            input data
        stock_dim : int
            number of unique stocks
        hmax : int
            maximum number of shares to trade
        initial_amount : int
            start money
        transaction_cost_pct: float
            transaction cost percentage per trade
        reward_scaling: float
            scaling factor for reward, good for training
        state_space: int
            the dimension of input features
        action_space: int
            equals stock dimension
        tech_indicator_list: list
            a list of technical indicator names
        turbulence_threshold: int
            a threshold to control risk aversion
        day: int
            an increment number to control date

    Methods
    -------
    _sell_stock()
        perform sell action based on the sign of the action
    _buy_stock()
        perform buy action based on the sign of the action
    step()
        at each step the agent will return actions, then 
        we will calculate the reward, and return the next observation.
    reset()
        reset the environment
    render()
        use render to return other functions
    save_asset_memory()
        return account value at each time step
    save_action_memory()
        return actions/positions at each time step
        

    """
    metadata = {'render.modes': ['human']}

    def __init__(self, 
                df,
                stock_dim,
                hmax,
                initial_amount,
                transaction_cost_pct,
                reward_scaling,
                state_space,
                action_space,
                tech_indicator_list,
                turbulence_threshold=None,
                lookback=24 * 30 * 3,
                day = 0):
        #super(StockEnv, self).__init__()
        #money = 10 , scope = 1
        self.day = day
        self.lookback=lookback
        self.df = df
        self.stock_dim = stock_dim
        self.hmax = hmax
        self.initial_amount = initial_amount
        self.transaction_cost_pct =transaction_cost_pct
        self.reward_scaling = reward_scaling
        self.state_space = state_space
        self.action_space = action_space
        self.tech_indicator_list = tech_indicator_list

        # action_space normalization and shape is self.stock_dim
        # self.action_space = spaces.Box(low = 0, high = 1,shape = (self.action_space,)) 
        self.action_space = spaces.Box(low = 0, high = 1,shape = (self.action_space,))
        # self.action_space = spaces.Box(low=0, high=1, shape=(self.stock_dim,), dtype=np.float32)
        # Shape = (34, 30)
        # covariance matrix + technical indicators
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape = (self.state_space+len(self.tech_indicator_list),self.state_space))

        # load data from a pandas dataframe
        self.data = self.df.loc[self.day,:]
        self.covs = self.data['cov_list'].values[0]
        self.state =  np.append(np.array(self.covs), [self.data[tech].values.tolist() for tech in self.tech_indicator_list ], axis=0)
        self.terminal = False     
        self.turbulence_threshold = turbulence_threshold        
        # initalize state: inital portfolio return + individual stock return + individual weights
        self.portfolio_value = self.initial_amount

        # memorize portfolio value each step
        self.asset_memory = [self.initial_amount]
        # memorize portfolio return each step
        self.portfolio_return_memory = [0]
        self.actions_memory=[[1/self.stock_dim]*self.stock_dim]
        self.date_memory=[self.data.date.unique()[0]]

        
    def step(self, actions):
        # Check if the episode is done
        self.terminal = self.day >= len(self.df.index.unique()) - 1
    
        if self.terminal:
            df = pd.DataFrame(self.portfolio_return_memory)
            df.columns = ['daily_return']
            plt.plot(df.daily_return.cumsum(), 'r')
            plt.savefig('results/cumulative_reward.png')
            plt.close()
    
            plt.plot(self.portfolio_return_memory, 'r')
            plt.savefig('results/rewards.png')
            plt.close()
    
            print("=================================")
            print("begin_total_asset:{}".format(self.asset_memory[0]))
            print("end_total_asset:{}".format(self.portfolio_value))
    
            df_daily_return = pd.DataFrame(self.portfolio_return_memory)
            df_daily_return.columns = ['daily_return']
            if df_daily_return['daily_return'].std() != 0:
                sharpe = ((24 * 30 * 3)**0.5) * df_daily_return['daily_return'].mean() / \
                         df_daily_return['daily_return'].std()
                print("Sharpe: ", sharpe)
            print("=================================")
    
            # Return observation, reward, terminated, truncated, and info
            return self.state, self.reward, self.terminal, False, {}
    
        else:
            # Normalize actions
            weights = self.softmax_normalization(actions)
            self.actions_memory.append(weights)
            last_day_memory = self.data
    
            # Load next state
            self.day += 1
            self.data = self.df.loc[self.day, :]
            self.covs = self.data['cov_list'].values[0]
            self.state = np.append(
                np.array(self.covs),
                [self.data[tech].values.tolist() for tech in self.tech_indicator_list],
                axis=0
            )
    
            # Calculate portfolio return
            portfolio_return = sum(((self.data.close.values / last_day_memory.close.values) - 1) * weights)
            new_portfolio_value = self.portfolio_value * (1 + portfolio_return)
            self.portfolio_value = new_portfolio_value
    
            # Save into memory
            self.portfolio_return_memory.append(portfolio_return)
            self.date_memory.append(self.data.date.unique()[0])
            self.asset_memory.append(new_portfolio_value)
    
            # Reward is the new portfolio value
            self.reward = new_portfolio_value
    
            # Return observation, reward, terminated, truncated, and info
            return self.state, self.reward, self.terminal, False, {}

    def reset(self, seed=None, options=None):
        # Если нужно, инициализируйте генератор случайных чисел
        if seed is not None:
            self._seed(seed)
        
        # Остальная логика вашего метода reset()
        self.asset_memory = [self.initial_amount]
        self.day = 0
        self.data = self.df.loc[self.day, :]
        self.covs = self.data['cov_list'].values[0]
        self.state = np.append(
            np.array(self.covs),
            [self.data[tech].values.tolist() for tech in self.tech_indicator_list],
            axis=0
        )
        self.portfolio_value = self.initial_amount
        self.terminal = False
        self.portfolio_return_memory = [0]
        self.actions_memory = [[1 / self.stock_dim] * self.stock_dim]
        self.date_memory = [self.data.date.unique()[0]]
        return self.state, {}  # Возвращайте состояние и пустой словарь (info)
    
    def render(self, mode='human'):
        return self.state
        
    def softmax_normalization(self, actions):
        numerator = np.exp(actions)
        denominator = np.sum(np.exp(actions))
        softmax_output = numerator/denominator
        return softmax_output

    
    def save_asset_memory(self):
        date_list = self.date_memory
        portfolio_return = self.portfolio_return_memory
        #print(len(date_list))
        #print(len(asset_list))
        df_account_value = pd.DataFrame({'date':date_list,'daily_return':portfolio_return})
        return df_account_value

    def save_action_memory(self):
        # date and close price length must match actions length
        date_list = self.date_memory
        df_date = pd.DataFrame(date_list)
        df_date.columns = ['date']
        
        action_list = self.actions_memory
        df_actions = pd.DataFrame(action_list)
        df_actions.columns = self.data.tic.values
        df_actions.index = df_date.date
        #df_actions = pd.DataFrame({'date':date_list,'actions':action_list})
        return df_actions

    def _seed(self, seed=None):
        self.np_random, seed = seeding.np_random(seed)
        return [seed]

    def get_sb_env(self):
        e = DummyVecEnv([lambda: self])
        obs = e.reset()
        return e, obs

In [ ]:
stock_dimension = len(train.tic.unique())
state_space = stock_dimension
print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")

In [15]:
from finrl import config
from finrl import config_tickers
# Create Folders
import os
if not os.path.exists("./" + config.DATA_SAVE_DIR):
    os.makedirs("./" + config.DATA_SAVE_DIR)
if not os.path.exists("./" + config.TRAINED_MODEL_DIR):
    os.makedirs("./" + config.TRAINED_MODEL_DIR)
if not os.path.exists("./" + config.TENSORBOARD_LOG_DIR):
    os.makedirs("./" + config.TENSORBOARD_LOG_DIR)
if not os.path.exists("./" + config.RESULTS_DIR):
    os.makedirs("./" + config.RESULTS_DIR)

In [ ]:
env_kwargs = {
    "hmax": 100, 
    "initial_amount": 1000000, 
    "transaction_cost_pct": 0.001, 
    "state_space": state_space, 
    "stock_dim": stock_dimension, 
    "tech_indicator_list": config.INDICATORS, 
    "action_space": stock_dimension, 
    "reward_scaling": 1e-4
    
}

e_train_gym = StockPortfolioEnv(df = train, **env_kwargs)

env_train, _ = e_train_gym.get_sb_env()
print(type(env_train))

## Implement DRL Algorithms

## Model 1: A2C

In [17]:
from finrl.agents.stablebaselines3.models import DRLAgent
agent = DRLAgent(env = env_train)

In [ ]:
agent = DRLAgent(env = env_train)

A2C_PARAMS = {"n_steps": 5, "ent_coef": 0.005, "learning_rate": 0.0002}
model_a2c = agent.get_model(model_name="a2c",model_kwargs = A2C_PARAMS)

trained_a2c = agent.train_model(model=model_a2c, 
                                tb_log_name='a2c',
                                total_timesteps=50000)

trained_a2c.save('./trained_models/trained_a2c.zip')

## Model 2: PPO

In [ ]:
agent = DRLAgent(env = env_train)
PPO_PARAMS = {
    "n_steps": 2048,
    "ent_coef": 0.005,
    "learning_rate": 0.0001,
    "batch_size": 128,
}
model_ppo = agent.get_model("ppo",model_kwargs = PPO_PARAMS)

trained_ppo = agent.train_model(model=model_ppo, 
                             tb_log_name='ppo',
                             total_timesteps=80000)

trained_ppo.save('./trained_models/trained_ppo.zip')

## Model 3: DDPG

In [ ]:
agent = DRLAgent(env = env_train)
DDPG_PARAMS = {"batch_size": 128, "buffer_size": 50000, "learning_rate": 0.001}
model_ddpg = agent.get_model("ddpg",model_kwargs = DDPG_PARAMS)

trained_ddpg = agent.train_model(model=model_ddpg, 
                             tb_log_name='ddpg',
                             total_timesteps=50000)

trained_ddpg.save('./trained_models/trained_ddpg.zip')

## Model 3: SAC

In [ ]:
import numpy as np
import pandas as pd
import gymnasium as gym
from stable_baselines3 import SAC 
from finrl.agents.stablebaselines3.models import DRLAgent


# Параметры, специфичные для ПОЛИТИКИ (архитектура сети)
POLICY_KWARGS = dict(net_arch=[256, 256])

# ОСНОВНЫЕ параметры модели SAC (все остальное)
MODEL_KWARGS = {
    "buffer_size": 100000,
    "batch_size": 256,
    "learning_rate": 0.0003,
    "gamma": 0.99,
    "tau": 0.005,
    "ent_coef": 'auto',
    "learning_starts": 10000
    # policy_kwargs УБРАН ОТСЮДА
}


model_sac = agent.get_model(
    "sac",
    policy_kwargs=POLICY_KWARGS, 
    model_kwargs=MODEL_KWARGS   
)


trained_sac = agent.train_model(
    model=model_sac,
    tb_log_name='sac',
    total_timesteps=100000
)


trained_sac.save('./trained_models/trained_sac.zip')

print("Модель SAC успешно создана, обучена и сохранена.")

## Trading

In [38]:
# https://github.com/Musonda2day/Asset-Portfolio-Management-usingDeep-Reinforcement-Learning-/blob/main/8.%20DRL_Portfolios.ipynb

In [33]:
# https://github.com/rathiromil13/DS-5500-Project-Portfolio-Optimization-Using-Deep-Reinforcement-Learning/blob/master/documents/Final_Report_Portfolio_Optimization_using_Deep_Reinforcement_Learning.pdf

In [26]:
import os
import pandas as pd

# Конфигурация
DATA_DIR = "../data"
MODELS_PREDICTION_DIR = os.path.join(DATA_DIR, "models_predictions")

# Создание папки для сохранения результатов
os.makedirs(MODELS_PREDICTION_DIR, exist_ok=True)

def evaluate_and_save_models(models, df, env_kwargs):
    """
    Оценивает модели и сохраняет результаты в CSV файлы.

    Параметры:
        models (dict): Словарь с моделями, где ключ - название модели, значение - объект модели.
        df (pd.DataFrame): Исходный DataFrame с данными.
        env_kwargs (dict): Параметры для создания среды StockPortfolioEnv.

    Возвращает:
        None
    """
    # Разделение данных на тестовый период
    trade_data = data_split(df, start_test, end_test)
    print(f"Тестовые данные: {trade_data.shape}")

    for model_name, model in models.items():
        print(f"Обработка модели: {model_name}")
        try:
            # Создание среды для торговли
            trade_env = StockPortfolioEnv(df=trade_data, **env_kwargs)

            # Получение предсказаний модели
            df_daily_return, df_actions = DRLAgent.DRL_prediction(
                model=model,
                environment=trade_env
            )

            # Убедимся, что столбец 'date' имеет формат datetime
            if 'date' in df_daily_return.columns:
                df_daily_return['date'] = pd.to_datetime(df_daily_return['date'])

            # Сохранение результатов
            save_model_predictions(
                df_daily_return=df_daily_return,
                df_actions=df_actions,
                model_name=model_name,
                data_dir=DATA_DIR
            )

        except Exception as e:
            print(f"Ошибка при обработке модели {model_name}: {e}")

def save_model_predictions(df_daily_return, df_actions, model_name, data_dir):
    """
    Сохраняет результаты предсказаний модели (доходность и действия) в CSV файлы.

    Параметры:
        df_daily_return (pd.DataFrame): DataFrame с ежедневной доходностью.
        df_actions (pd.DataFrame): DataFrame с действиями модели.
        model_name (str): Название модели (используется в имени файла).
        data_dir (str): Путь к папке data.

    Возвращает:
        None
    """
    # Путь к папке для сохранения файлов
    models_prediction_dir = os.path.join(data_dir, "models_predictions")

    # Имена файлов
    return_file_name = os.path.join(models_prediction_dir, f"{model_name}_return_series.csv")
    actions_file_name = os.path.join(models_prediction_dir, f"{model_name}_actions.csv")

    try:
        # Сохранение DataFrame с ежедневной доходностью
        df_daily_return.to_csv(return_file_name, index=False)
        print(f"Файл '{return_file_name}' успешно сохранен.")

        # Сохранение DataFrame с действиями модели
        df_actions.to_csv(actions_file_name, index=True)
        print(f"Файл '{actions_file_name}' успешно сохранен.")

    except Exception as e:
        print(f"Ошибка при сохранении файлов для модели {model_name}: {e}")

In [ ]:
models = {
    "model_a2c": model_a2c,
    "model_ppo": model_ppo,
    "model_ddpg": model_ddpg,
    "model_sac": model_sac
}


# Вызов функции для оценки и сохранения результатов моделей
evaluate_and_save_models(models=models, df=df, env_kwargs=env_kwargs)